<a href="https://colab.research.google.com/github/hrishiNavkar/SimpleLinearRegrMLmodel/blob/master/FuelEfficiencyAutomobiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

"""
@author: Hrishikesh Navkar
"""


In [1]:
#Importing required Packages
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [3]:
#importing dataset using keras from URL
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path


32768/30286 [================================] - 0s 2us/step


'/root/.keras/datasets/auto-mpg.data'

In [4]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']


In [9]:
#reading the dataset using pandas lib 
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)


In [10]:
#making copy of orogonal dataset

dataset = raw_dataset.copy()
dataset.tail()



,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [11]:
#dropping null values from dataset
dataset.isna().sum()
dataset = dataset.dropna()


In [12]:
#one hot encoding : "replacing single categoical column with seperate categoical column"
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
393,27.0,4,140.0,86.0,2790.0,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130.0,24.6,82,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295.0,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625.0,18.6,82,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720.0,19.4,82,1.0,0.0,0.0


In [13]:
#seperating dataset for Training & Testing purpose
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)


In [20]:
#describing traning dataset
train_stats = train_dataset.describe()
train_stats


,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
count,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000
mean,5.477707,195.318471,104.869427,2990.251592,15.559236,75.898089,0.624204,0.178344,0.197452
std,1.699788,104.331589,38.096214,843.898596,2.789230,3.675642,0.485101,0.383413,0.398712
min,3.000000,68.000000,46.000000,1649.000000,8.000000,70.000000,0.000000,0.000000,0.000000
25%,4.000000,105.500000,76.250000,2256.500000,13.800000,73.000000,0.000000,0.000000,0.000000
50%,4.000000,151.000000,94.500000,2822.500000,15.500000,76.000000,1.000000,0.000000,0.000000
75%,8.000000,265.750000,128.000000,3608.000000,17.200000,79.000000,1.000000,0.000000,0.000000
max,8.000000,455.000000,225.000000,5140.000000,24.800000,82.000000,1.000000,1.000000,1.000000


In [ ]:
#removing 'MPG' column from both train and test dataset 
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')


In [34]:
#taking taking transpose of train dataset to view mean, standard deviation of all columns
train_stats = train_stats.transpose()
train_stats


,count,mean,std,min,25%,50%,75%,max
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


In [35]:
#normalizing the data
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

In [36]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)


In [37]:
normed_train_data

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
146,-0.869348,-1.009459,-0.784052,-1.025303,-0.379759,-0.516397,0.774676,-0.465148,-0.495225
282,-0.869348,-0.530218,-0.442811,-0.118796,0.624102,0.843910,0.774676,-0.465148,-0.495225
69,1.483887,1.482595,1.447140,1.736877,-0.738281,-1.060519,0.774676,-0.465148,-0.495225
378,-0.869348,-0.865687,-1.099044,-1.025303,-0.308055,1.660094,0.774676,-0.465148,-0.495225
331,-0.869348,-0.942365,-0.994047,-1.001603,0.875068,1.115971,-1.286751,-0.465148,2.012852
...,...,...,...,...,...,...,...,...,...
281,0.307270,0.044872,-0.521559,-0.000298,0.946772,0.843910,0.774676,-0.465148,-0.495225
229,1.483887,1.961837,1.972127,1.457223,-1.598734,0.299787,0.774676,-0.465148,-0.495225
150,-0.869348,-0.836932,-0.311564,-0.710099,-0.021237,-0.516397,-1.286751,-0.465148,2.012852
145,-0.869348,-1.076553,-1.151543,-1.169870,1.233589,-0.516397,-1.286751,-0.465148,2.012852


In [39]:
#building Model
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]), #input layer 
    layers.Dense(64, activation='relu'), #hidden layer
    layers.Dense(1) # output layer
  ])
  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [40]:
model = build_model()
#describing the properties of Model
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                640       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________


compiling the model

In [41]:
EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS)


Epoch 1/1000
10/10 [==============================] - 0s 2ms/step - loss: 572.3509 - mae: 22.7068 - mse: 572.3509
Epoch 2/1000
10/10 [==============================] - 0s 2ms/step - loss: 507.8168 - mae: 21.3574 - mse: 507.8168
Epoch 3/1000
10/10 [==============================] - 0s 1ms/step - loss: 440.0576 - mae: 19.8229 - mse: 440.0576
Epoch 4/1000
10/10 [==============================] - 0s 2ms/step - loss: 364.6410 - mae: 17.9487 - mse: 364.6410
Epoch 5/1000
10/10 [==============================] - 0s 1ms/step - loss: 288.0443 - mae: 15.7583 - mse: 288.0443
Epoch 6/1000
10/10 [==============================] - 0s 1ms/step - loss: 213.6987 - mae: 13.2779 - mse: 213.6987
Epoch 7/1000
10/10 [==============================] - 0s 2ms/step - loss: 148.8713 - mae: 10.6234 - mse: 148.8713
Epoch 8/1000
10/10 [==============================] - 0s 1ms/step - loss: 99.5325 - mae: 8.3186 - mse: 99.5325
Epoch 9/1000
10/10 [==============================] - 0s 1ms/step - loss: 68.2753 - mae: 6.

In [46]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)#checking loss, mean absoulute error and mean standard error

In [51]:
#predicting the required values of MPG
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

array([13.865457,  8.850942, 11.751533, 23.28061 , 19.512812, 13.453529,
       13.907723, 12.744496, 20.346579, 30.580538, 23.814919, 22.054039,
       16.014767, 25.162683, 12.669178, 15.366698, 14.413033, 13.383066,
       18.575748, 12.911143, 14.137478, 25.300856, 18.364492, 20.675884,
       29.704409, 23.332045, 16.886518, 23.204327, 18.862062, 17.605425,
       25.45651 , 22.20466 , 19.358011, 20.43721 , 25.58673 , 15.105868,
       20.41818 , 26.660742, 28.917397, 18.06352 , 29.643095, 27.402355,
       15.061734, 38.57476 , 35.981598, 37.52648 , 19.957743, 20.910866,
       21.046055, 23.31119 , 31.81978 , 16.11822 , 33.52958 , 19.020212,
       18.330542, 18.886686, 34.227844, 36.57058 , 21.345684, 25.029718,
       42.7706  , 34.15464 , 28.126928, 25.939474, 32.635796, 39.103638,
       36.95192 , 36.71093 , 37.623028, 32.753883, 22.03744 , 32.593735,
       29.435122, 30.25545 , 39.706856, 34.65975 , 37.739994, 29.74317 ],
      dtype=float32)

In [49]:
test_labels #comparing to this orignal values of MPG

9      15.0
25     10.0
28      9.0
31     25.0
33     19.0
       ... 
369    34.0
375    36.0
382    34.0
384    32.0
396    28.0
Name: MPG, Length: 78, dtype: float64

saving the model in tflite format 

In [52]:
kearas_file = "automobile.h5"
tf.keras.models.save_model(model,kearas_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model)


In [53]:
tfmodel = converter.convert()
open("automobile.tflite","wb").write(tfmodel)

20696